In [64]:
import numpy as np
import matplotlib.pyplot as plt

import unicodedata
import codecs
import re
import os.path
import string
#import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
#nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer

# Chargement des données

In [2]:
def load_movies(path2data): # 1 classe par répertoire
    alltxts = [] # init vide
    labs = []
    cpt = 0
    for cl in os.listdir(path2data): # parcours des fichiers d'un répertoire
        #print(cl)
        #print(type(cl))
        if cl == '.DS_Store':
            continue
        for f in os.listdir(path2data+cl):
            txt = open(path2data+cl+'/'+f).read()
            alltxts.append(txt)
            labs.append(cpt)
        cpt+=1 # chg répertoire = cht classe

    return alltxts,labs

In [15]:
path = "./Dataset/movies1000/"

alltxts,alllabs = load_movies(path)

print(alltxts[10])
print(alllabs[10])

starring shawnee smith ; donovan leitch ; ricky paull goldin ; kevin dillon & billy beck the blob is the remake of the 1960's classic ( a term that i use very loosely to define the original ) about a really mean glob of goop that takes out anything that gets in its way . 
now the original version has the virtue of cheesy special effects which give it a kind of nostalgic campy feel . 
the fact that steve mcqueen was the star of the film doesn't exactly hurt it either . 
fast forward to the late '80's . 
steve mcqueen isn't in the remake ( this might have something to do with the fact that he had been pushing up daisies for years ) . 
nor is there any hollywood heavy hitters in the lead role . 
that is unless you count matt dillon's brother kevin as a hollywood heavy hitter . 
the other thing that works against this new blob is that special effects technology has improved dramatically since the original . 
the only things you need now to have great special effects in a movie are deep poc

# Pretraitement

In [66]:
def keep_only_part(txt_list, number):
    #A utiliser AVANT remove_ponctuation() sinon on peut pas reconnaitre les lignes

    if number is None:
        return txt_list
        
    tmp = txt_list
    for i in range(len(tmp)):
        tmp[i] = tmp[i].split('\n')
        if number > len(tmp[i]):
            print(f"error number{number} greater then length of text{len(tmp[i])}")
            return
        if number > 0:
            tmp[i] = '\n'.join(tmp[i][:number])
        else:
            tmp[i] = '\n'.join(tmp[i][number:])

    return tmp
    

def remove_caps(txt_list):

    tmp = txt_list
    for i in range(len(txt_list)):
        tmp[i].lower()

    return tmp

def remove_ponctuation(txt_list):

    tmp = txt_list
    punc = string.punctuation
    #print(punc)
    punc += '\n\r\t'
    for i in range(len(txt_list)):
        #tmp[i] = re.sub(r"\b's\b", '', tmp[i])
        #tmp[i] = tmp[i].translate(str.maketrans(punc, ' ' * len(punc)))
        tmp[i] = tmp[i].translate(str.maketrans('', '', punc))

    return tmp

def stemming(txt_list):

    ps = PorterStemmer()
    tmp = txt_list
    for i in range(len(txt_list)):
        tmp[i] = ' '.join([ps.stem(word) for word in tmp[i].split()])
    #test = ps.stem("isnt")
    #print(test)
    return tmp
    
def change_capital_words(txt_list):

    tmp = txt_list
    for i in range(len(txt_list)):
        #if re.findall(r'\b[A-Z]+(?:\s+[A-Z]+)*\b', tmp[i]):
            #print("title found", re.findall(r'\b[A-Z]+(?:\s+[A-Z]+)*\b', tmp[i]))
        tmp[i] = re.sub(r'\b[A-Z]+(?:\s+[A-Z]+)*\b', 'TITLE',tmp[i])

    return tmp

def remove_numbers(txt_list):

    tmp = txt_list
    for i in range(len(tmp)):

        tmp[i] = re.sub('[0-9]+', '', tmp[i])

    return tmp

def vectorizer(txt_list, language):

    assert (language == 'FRENCH' or language == 'ENGLISH'), "Language value needs to be either FRENCH or ENGLISH"
    
    if language == "FRENCH":
        stop_list = stopwords.words('french')
    elif language == "ENGLISH":
        stop_list = stopwords.words('english')

    vectorizer = CountVectorizer(stop_words=stop_list)
    X = vectorizer.fit_transform(txt_list)

    return X, vectorizer

#print(alltxts[9])
text = keep_only_part(alltxts[:100], None)
#print(text)
text = remove_caps(text)
text = remove_ponctuation(text)
text = change_capital_words(text)
text = remove_numbers(text)
text = stemming(text)
X, vectorizer = vectorizer(text, "ENGLISH")
print(vectorizer.get_feature_names_out())
print(X.toarray())
#print(text[9])

['aaa' 'aaliyah' 'aalyah' ... 'zoo' 'zoom' 'zoot']
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
